In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib.layers.python.layers import layers as layers_lib
from tensorflow.python.ops import array_ops

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

tf.logging.set_verbosity(tf.logging.INFO)

# GPU selection (memory)
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [ ]:
# GPU selection (memory)
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
data_txt = open('./data_split/5fold_0802.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [ ]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 224
r_h = 224

total_pix = r_w * r_h * channel

In [ ]:
def vgg16(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, height, width, channels]
    # Our Fishing net image size is 640x480 and 3-channel (RGB)
    input_layer = tf.reshape(features["x"], [-1, 224, 224, 3])
    dropout_keep_prob = 0.5
    num_classes = 2
    is_training = True
    
    
    net = layers_lib.repeat(
        input_layer, 2, layers.conv2d, 64, [3, 3], scope='conv1')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool1')
    net = layers_lib.repeat(net, 2, layers.conv2d, 128, [3, 3], scope='conv2')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool2')
    net = layers_lib.repeat(net, 3, layers.conv2d, 256, [3, 3], scope='conv3')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool3')
    net = layers_lib.repeat(net, 3, layers.conv2d, 512, [3, 3], scope='conv4')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool4')
    net = layers_lib.repeat(net, 3, layers.conv2d, 512, [3, 3], scope='conv5')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool5')
    # Use conv2d instead of fully_connected layers.
    net = layers.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
    net = layers_lib.dropout(
        net, dropout_keep_prob, is_training=is_training, scope='dropout6')
    net = layers.conv2d(net, 4096, [1, 1], scope='fc7')
    net = layers_lib.dropout(
        net, dropout_keep_prob, is_training=is_training, scope='dropout7')
    net = layers.conv2d(
        net,
        num_classes, [1, 1],
        activation_fn=None,
        normalizer_fn=None,
        scope='fc8')
    net = array_ops.squeeze(net, [1, 2], name='fc8/squeezed')
    
    logits = net.layers.dropout(dense1, rate=dropout, training=is_training, name='drop')
        
    # Output
    out = tf.layers.dense(dropout, n_class, name='output')
    out = tf.nn.softmax(out) if not is_training else out

    return out

In [ ]:
dropout = 0.5
learning_rate = 0.0001
batch_size = 16
disp_step = 20
save_step = 500
n_classes = 2
epochs = 10

tr_steps = int(tr_data.shape[0]/batch_size*epochs)
inner_steps = int((tr_data.shape[0])/batch_size)

In [ ]:
with tf.device('/cpu:0'):
    reuse_vars = False
    
    # tf graph input
    X = tf.placeholder(tf.float32, [None, total_pix], name='inputs')
    Y = tf.placeholder(tf.int32, [None, 1], name='labels')
    
    with tf.device('/gpu:0'):
        # cnn_model(x, n_class, dropout, re_use, is_training, width, height)
        logits_train = cnn_model(X, n_classes, dropout, reuse=reuse_vars,
                                 is_training=True)
        
        logits_test = cnn_model(X, n_classes, dropout, reuse=True,
                                is_training=False)
        
        loss = tf.losses.sparse_softmax_cross_entropy(logits=logits_train, labels=Y)
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        grads = optimizer.compute_gradients(loss)
        
        pred = tf.argmax(input=logits_test, axis=1)
        correct_pred = tf.equal(pred, tf.cast(Y, tf.int64))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        #accuracy = tf.metrics.accuracy(labels=Y, predictions=pred)
        
        
        prob = tf.nn.softmax(logits_test, name='softmax_tensor')
        
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        
        reuse_vars = True
        
    
    
    
    
    saver = tf.train.Saver()
    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()

In [ ]:
    with tf.Session() as sess:

        sess.run(init_g)
        sess.run(init_l)

        t_count = 1
        saver.save(sess, './models/test')
        for j in range(epochs):

            for i in range(inner_steps):

                tr_batch = {}

                if i < inner_steps-1:
                    tr_batch['data'] = tr_data[i*batch_size:(i+1)*batch_size][:]
                    tr_batch['label'] = tr_label[i*batch_size:(i+1)*batch_size][:]

                else:
                    tr_batch['data'] = tr_data[i*batch_size:][:]
                    tr_batch['label'] = tr_label[i*batch_size:][:]

                sess.run(train_op, feed_dict={X: tr_batch['data'],
                                              Y: tr_batch['label']})

                if t_count % disp_step == 1:
                    
                    loss_tr = sess.run(loss, feed_dict = {X: tr_batch['data'],
                                                          Y: tr_batch['label']})

                    acc_tr = sess.run(accuracy, feed_dict = {X: tr_batch['data'],
                                                             Y: tr_batch['label']})
                    
                    prob_tr = sess.run(prob, feed_dict = {X: tr_batch['data'],
                                                         Y: tr_batch['label']})

                    #print('step %d, training accuracy %f, loss %f' % (t_count, accuracy, loss))
                    print('Training step: ', t_count)
                    print('Accuracy', acc_tr)
                    print('Probability')
                    print(prob_tr)
                    print('Loss: ', loss_tr)
                    #print(acc_tr, loss_tr)
                    
                if t_count % save_step == 0:
                    saver.save(sess, './models/test', t_count)

                t_count = t_count+1
        
        saver.save(sess, './models/test', t_count)
        print("Training is finished!!!")
        
        
        
        ev_accuracy = np.zeros((len(ev_label), 1))
        for k in range(len(ev_label)):
            ev_batch = {}
            ev_batch['data'] = ev_data[k:(k+1)][:]
            ev_batch['label'] = ev_label[k:(k+1)]
            ev_accuracy[k] = sess.run(accuracy, feed_dict={X: ev_batch['data'],
                                                           Y: ev_batch['label']})

        print(np.mean(ev_accuracy))